# N-Grams

Adapted from https://github.com/PradipKumarChaudhary1/N-gram-model


In [23]:
import numpy as np
import os
import textwrap
import urllib

In [24]:
first_possible_word = {}
second_possible_word = {}
transition = {}


def expandDict(
    dictionary, key, value
):  # storing into dictionary current word as a 'value' and previous word as a 'key'
    if key not in dictionary:
        dictionary[key] = []
    dictionary[key].append(value)


def get_next_probability(word_list):  # finding probability of each word
    word_list_length = len(word_list)
    probability_dict = {}
    for item in word_list:
        probability_dict[item] = (
            probability_dict.get(item, 0) + 1
        )  # calculating frequncy of any word
    for (
        key,
        value,
    ) in probability_dict.items():  # calculating probability and store into dictionary according to thire key and probability as a value
        probability_dict[key] = (value + 1) / (word_list_length + 9484)
    probability_dict = sort_prob(probability_dict)
    return probability_dict


def sort_prob(dictionary):  # sorting of dictionary through values
    keys_list = list(dictionary.keys())
    values_list = list(dictionary.values())
    for i in range(len(values_list) - 1):
        for j in range(len(values_list) - i - 1):
            if values_list[j] < values_list[j + 1]:
                temp = values_list[j]
                values_list[j] = values_list[j + 1]
                values_list[j + 1] = temp
                temp = keys_list[j]
                keys_list[j] = keys_list[j + 1]
                keys_list[j + 1] = temp

    sort_dict = {}
    for i in range(len(values_list)):
        sort_dict[keys_list[i]] = values_list[i]
    return sort_dict


def trainModel(filepath):
    for line in open(filepath):
        tokens = line.rstrip().lower().split()
        tokens_length = len(tokens)
        for i in range(tokens_length):
            token = tokens[i]
            if i == 0:  # if word is ffirst word of every sentence
                first_possible_word[token] = first_possible_word.get(token, 0) + 1
            else:
                prev_token = tokens[i - 1]
                if i == 1:  # if word is 2nd word of the senetnce
                    expandDict(second_possible_word, prev_token, token)
                if i == tokens_length - 1:  # if word is last of sentence
                    expandDict(transition, (prev_token, token), "END")
                else:
                    prev_prev_token = tokens[i - 2]
                    expandDict(transition, (prev_prev_token, prev_token), token)

    first_possible_word_total = sum(
        first_possible_word.values()
    )  # finding total frequency n first_possible word
    for (
        key,
        value,
    ) in first_possible_word.items():  # calculating probability of first word in each sentence and store according to that word
        first_possible_word[key] = (value + 1) / (first_possible_word_total + 9484)

    for prev_word, next_word_list in second_possible_word.items():
        second_possible_word[prev_word] = get_next_probability(next_word_list)

    for word_pair, next_word_list in transition.items():
        transition[word_pair] = get_next_probability(next_word_list)


def next_word_one(tpl):
    d = second_possible_word.get(
        tpl
    )  # tpl match with key and return value, value is already dictionary
    if d is not None:
        return list(d.keys())[:5]  # since d is already dictionary so return keys
    else:
        probs = np.array(list(first_possible_word.values()))
        probs = probs / np.sum(probs)  # Make a probability distribution (sum to one)
        return np.random.choice(
            a=list(first_possible_word.keys()), size=5, p=probs
        ).tolist()


def next_word(tpl):
    if type(tpl) is str:  # it is first word of string. return from second word
        return next_word_one(tpl.lower())
    if (
        type(tpl) is tuple
    ):  # incoming words are combination of two words. find next word now based on transitions
        d = transition.get(tpl)
        if d is None:
            return next_word_one(tpl[1])
        return list(d.keys())[:5]
    return None  # wrong input.. return nothing

In [25]:
# Make sure that the path is correct.
# curDir = os.path.join(os.getcwd(), "T10 - LLM", "S01 - Ngram")
curDir = os.path.join(os.getcwd())
filePath = os.path.join(curDir, "training_text.txt")
print(filePath)

if not os.path.exists(filePath):
    
    target_url = "https://github.com/ie-ai-class/ai-2567-2/raw/refs/heads/main/T10%20-%20LLM/S01%20-%20N-Grams/training_text.txt"
    data = urllib.request.urlopen(target_url)

    with open(filePath, "wb") as file:
        for line in data:
            file.write(line)

    with open(filePath) as file:
        text = file.read().lower()

c:\Users\nnnpo\Coding\ai-class\ai-2567-2\T10 - LLM\S01 - N-Grams\training_text.txt


In [26]:
# Train model
trainModel(filePath)

## Inspecting `first_possible_word`


In [27]:
for idx, (k, v) in enumerate(first_possible_word.items()):
    print(f"{k:5s}: {v:5.4f}")
    if idx > 5:
        break

to   : 0.0013
i    : 0.0512
in   : 0.0042
it   : 0.0254
all  : 0.0014
he   : 0.0148
they : 0.0035


## Inspecting `second_possible_word`


In [28]:
for idx1, (k1, v1) in enumerate(second_possible_word.items()):
    print(k1)
    for idx2, (k2, v2) in enumerate(v1.items()):
        print(f"\t{k2:5s}: {v2:5.4f}")
        if idx2 > 1:
            break
    if idx1 > 3:
        break

to
	me   : 0.0005
	sherlock: 0.0002
	speak: 0.0002
i
	have : 0.0077
	am   : 0.0059
	had  : 0.0037
in
	the  : 0.0021
	a    : 0.0008
	this : 0.0007
it
	is   : 0.0148
	was  : 0.0105
	must : 0.0013
all
	the  : 0.0003
	was  : 0.0003
	day  : 0.0003


## Inspecting `transition`


In [29]:
for idx1, (k1, v1) in enumerate(transition.items()):
    print(k1)
    for idx2, (k2, v2) in enumerate(v1.items()):
        print(f"\t{k2:5s}: {v2:5.4f}")
        if idx2 > 1:
            break
    if idx1 > 3:
        break

('woman', 'to')
	sherlock: 0.0002
	him  : 0.0002
	bear : 0.0002
('to', 'sherlock')
	holmes: 0.0003
('sherlock', 'holmes')
	END  : 0.0010
	was  : 0.0010
	s    : 0.0008
('holmes', 'she')
	cried: 0.0003
	is   : 0.0002
	used : 0.0002
('she', 'is')
	a    : 0.0004
	not  : 0.0004
	now  : 0.0004


## Generating text


In [48]:
startWord = "I"
wordLength = 300

wordList = [startWord]
for i in range(wordLength):
    if len(wordList) == 1:
        res = next_word(wordList[0])
    else:
        res = next_word((wordList[-2], wordList[-1]))
    wordList.append(res[0])

longStr = " ".join(wordList).replace(" END ", ". ")
print(textwrap.fill(longStr, width=80))

I have you not find me ungrateful holmes turned to the other. he was a very
serious one to the other. has only been here. i have no doubt that the doctor s
advice and help that is the first place we may take it that it was a very
serious one to the other. a few minutes with his head sunk upon his face. your
majesty. its disappearance however was but one way out of the most perfect
reasoning and observing machine that the doctor s advice and help that is the
first place we may take it that it was a very serious one to the other. i have
no doubt that the doctor s advice and help that is the first place we may take
it that it was a very serious one to the other. it is a very serious one to the
other. he was a very serious one to the other. by the fire. i have no doubt that
the doctor s advice and help that is the first place we may take it that it was
a very serious one to the other. it is a very serious one to the other. and now
i am sure that i have no doubt that the doctor s advice and